<a href="https://colab.research.google.com/github/orionhunts-ai/new_models_datasets/blob/main/morpheus_cyber_ft_gpt4o_mini_models-processed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
from loguru import logger

RUNS= 0
if RUNS == 0:
  # @title Install Core Libraries { run: "auto", display-mode: "form" }
  %pip -qqq install -U loguru uuid typing requests pyarrow
  %pip install -qqq torch
  ###LOGURU CONFIG### ALEX NOTE: DEPENDENCY BUT THIS IS NOT PROD
  _logger = logger
  logger.log_level = "DEBUG"
  _logger.info("__Hello Human__")
  RUNS += 1
else:
  pass



RUN=0
if RUN == 0:
  try:
    !pip install -qqq -U torch
    !pip -qqq install transformers datasets wandb
    _logger.info("Installed libs")
    RUNS += 1
  except Exception as e:
    _logger.error(e)
    RUNS==0
    pass
else:
  pass



2024-07-26 21:50:23.923 | INFO     | __main__:<cell line: 5>:12 - __Hello Human__


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.15 requires torch<2.4,>=1.10, but you have torch 2.4.0 which is incompatible.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.4.0 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.4.0 which is incompatible.


2024-07-26 21:51:09.482 | INFO     | __main__:<cell line: 20>:24 - Installed libs


# Fine Tuning OpenAI GPT-4o-Mini For (Free) Agentic

*   476 Training Examples
*   Cyber focused to assist with labelling threats

Cyber 👾

  *  Training (and reporting) on Google Colab
   for access to their high powered CUDA and  
  Leveraging
    * fine tuning on dataset ```"swaption2009/cyber-threat-intelligence-custom-data”```
    * Open AI offering this mini version of the already efficient gpt4o.
        * OAI claims that the mini is almost as performant (but it's 20x cheaper)
    * Sampling from the full set for those that are most relevant to Cyber Security Analysts.***

    * Aside from traditional and evolving Evaluations I will also deploy a number of the finely tuned models in a Microsoft Autogen agentic environment to see how they perform on basic analysis on a database.

    * ```Red Panda``` (a high performance streaming data alternative to ```Kafka``` will be used)



## What about Phi!?!
The last experiment with Phi is still ongoing. I am having CUDA compatibility issues between the librares and it's a good chance to learn a bit deeper into that stack


##  Data and Tool Preparation
**Summary:**

This study explores the fine-tuning of the Phi-3-small-instruct model (7.39 billion parameters) by using Daniel So's Unsloth for a Cyber Threat Intelligence (CTI) task using methods like Parameter-Efficient Fine-Tuning (PEFT), Low-Rank Adaptation (LoRA), and Quantized Low-Rank Adaptation (QLoRA). It aims to evaluate performance degradation, model collaboration in agentic environments, and the potential influence of GPT-4. Synthetic data from gretel.ai was also utilized to supplement the fine-tuning process and enhance data diversity and robustness.

In [5]:
from google.colab import userdata
VERSION = userdata.get('__VERSION__')
BASE_URL = userdata.get('__BASE_URL__')
os.environ["WANDB_MODE"] = "enabled"
pretrained = "gpt4o-mini"
project_type = "fine_tuning"
orion_type = "cyber"
model_name = f'{project_type}-{pretrained}-{orion_type}.Vx0'
DATASET = "swaption2009/cyber-threat-intelligence-custom-data"

os.chdir(BASE_URL)
if project_type == None:
  project_type = "snyata"
else:
  project_type=project_type

project_name = f"orion-intel-{pretrained}-{project_type}-x0"
model_name = project_name

_project_stats = (f'''
                  #### HELLO_HUMAN_ ######
                  RUNS: {RUNS}
                  PWD: {os.getcwd()}
                  VERSION: {VERSION}
                  PROJECT: {project_name}
                  BASE_URL: {BASE_URL}
                  DATA_URL  f'./data/'
                  MODEL_URL: f'./models/'
                  PRETRAINED: {pretrained}
                  PROJECT_TYPE: {project_type}
                  ########
                  PROJECT NAME: {project_name}
                  MODEL_NAME:  {model_name}
                  DATASET: {DATASET} ''')
_logger.info=_project_stats
print(_project_stats)


                  #### HELLO_HUMAN_ ######
                  RUNS: 2
                  PWD: /content/drive/MyDrive/models_datasets
                  VERSION: 0.1.0
                  PROJECT: orion-intel-gpt4o-mini-fine_tuning-x0
                  BASE_URL: /content/drive/MyDrive/models_datasets/
                  DATA_URL  f'./data/'
                  MODEL_URL: f'./models/'
                  PRETRAINED: gpt4o-mini
                  PROJECT_TYPE: fine_tuning
                  ########
                  PROJECT NAME: orion-intel-gpt4o-mini-fine_tuning-x0
                  MODEL_NAME:  orion-intel-gpt4o-mini-fine_tuning-x0
                  DATASET: swaption2009/cyber-threat-intelligence-custom-data 


In [11]:
# IMPORTS
import os
import random
import pandas as pd
import numpy as np
import torch
import wandb
from typing import List, Dict, Union, Optional
from tqdm import tqdm
import matplotlib.pyplot as plt
import huggingface_hub
from huggingface_hub import notebook_login
from google.colab import userdata
from transformers import TrainingArguments, Trainer
import pyarrow
import random
!wandb login $WANDB_API_KEY

# WANDB CONFIG
os.environ["WANDB_MODE"]= "online"
os.environ["WANDB_PROJECT"] = project_name
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')
os.environ["WANDB_NOTEBOOK_NAME"] = f"{os.path.dirname(os.path.abspath(os.path.expanduser(__name__)))}"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
hf_hub_key=userdata.get('HF_HUB_WRITE')
wandb_key=userdata.get('WANDB_API_KEY')
huggingface_hub.login(token=hf_hub_key, add_to_git_credential=True, write_permission=True)
#notebook_login(new_session=False)

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [12]:
def wandb_run(name, project_type, project=project_name):
  try:
    if wandb.run is None and os.environ["WANDB_MODE"] != "disabled":
      wandb.init(project=project, job_type=project_type, name=name)
      project_init = wandb.init(project=project_name, job_type= "data_preparation" ,dir=f"{BASE_URL}{project_name}/")
      _logger.info(f"###WANDB_RUN:{wandb.run.name}###")
    elif os.environ["WANDB_MODE"] == "disabled":
      _logger.info(f"##### WANDB DISABLED ### ")

  except Exception as e:
    logger.error(e, exc_info=True)



In [13]:
####PROJECT DEFINITION######
model_prov = {"openai": f"gpt4o-mini",
               "mistral": f"mistral-7b",
               "orion": f"orionai-cyber-{orion_type}-x0"}

model_prov=model_prov.get("orionai")
model_prov

args = TrainingArguments(
    # other args and kwargs here
    report_to="wandb",  # enable logging to W&B
    run_name="pre-processing-cyber",  # name of the W&B run (optional)
    logging_steps=5,
    output_dir=BASE_URL)

if wandb.run is None:
  _logger.error("""#### WANDB DISABLED """)
  wandb_enable = input("Do you want to enable Weights and Biases?")
  if wandb_enable == "yes" or "y":
    os.environ["WAND_MODE"]="enabled"
    wandb.init(project=project_name, job_type="data_preparation", dir=f"{BASE_URL}{project_name}/")
  else:
    logger.info(wandb.mode)
else:
    logger.info(f"""
                ########## WANDB_RUN ########
                RUNS: {RUNS}
                NAME: {wandb.run.name}
                OUTPUT_DIR: {BASE_URL}{project_name}/
                MODEL: {model_prov}
                DATASET: {DATASET}
                ####### ###########
                  """)


2024-07-26 21:53:11.524 | ERROR    | __main__:<cell line: 16>:17 - #### WANDB DISABLED 


Do you want to enable Weights and Biases?


wandb: Currently logged in as: orionhunts (synavatelabs). Use `wandb login --relogin` to force relogin
wandb: WARNING Path /content/drive/MyDrive/models_datasets/orion-intel-gpt4o-mini-fine_tuning-x0/wandb/ wasn't writable, using system temp directory.


# The BSNS

In [14]:
class WandBArtifact():
  def __init__(self, artifact_name, data, run=wandb.run, type=Union["data", "model","table"]):
    self.run = run
    self.artifact = artifact
    self.type = type

    if self.type == "data":
      artifact = wandb.Table(dataframe=artifact_name, columns=[col for col in data.columns])

    # Add the table to an Artifact to increase the row
    # limit to 200000 and make it easier to reuse
    new_artifact = wandb.Artifact(f"{project}_{artifact_name}-{self.type}", type=self.type)
    artifact.add(data, artifact_name)

    # log the raw csv file within an artifact to preserve our data
    artifact.add_file(f"{BASE_URL}/{model_name}.csv")

    # Start a W&B run to log data
    run = f"{random.randint(0,50)}-run"
    if wandb.run != run:
      wandb.init(project=project, job_type="data_preparation", name=run)
    # Log the table to visualize with a run...
    run.log({"artifact_name": f"{artifact_name}"})
    run.log({"job_type": f"{job_type}"})
    run.save()

    # and Log as an Artifact to increase the available row limit!
    run.log_artifact(new_artifac)
    run.save()

    _logger.info(f"{artifact} MADE\n")

In [15]:
#ML Ops and EDA Imports
import os
import torch
import pandas as pd
import numpy as np
from google.colab import userdata
from huggingface_hub import notebook_login

# Retrieve API keys from user data


# Login to Weights & Biases
'''if wandb_key:
    wandb.login(key=wandb_key)
else:
    print("WANDB_API_KEY is not set")

Login to Hugging Face
if hf_token:
    os.system(f"huggingface-cli login --token {hf_token} --add_to_git_credential")
    os.system(f"huggingface-hub login --token {hf_token}")
else:
     print("HF_TOKEN is not set")'''

# Check if cuda on Colab
device = "###### DEVICE IS:  cuda:0" if torch.cuda.is_available() else "cpu"
#logger.info(device)

In [ ]:
# Make a repo
new_repo = False
name="morpheus_cyber_gpt4o-mini"
if new_repo == False or huggingface_hub.repo_exists(repo_id=name):
  pass
  _logger.info(f"Repo {name} already exists")
else:
  huggingface_hub.create_repo(repo_id=name)
  _logger.info(f"Created repo {name}")

In [16]:
# Loading pre-determined Cyber Data - Create your own Synthetic data on top at https://gretel.ai/'''
import pandas as pd
try:
  from datasets import load_dataset
  ds = load_dataset("swaption2009/cyber-threat-intelligence-custom-data")
  logger.debug(ds)
  df_train = ds['train'].to_pandas()
  logger.debug(df_train[0:10])
  logger.debug(type(df_train))
  logger.debug(df_train.head())
  logger.info({df_train[0]})

except Exception as e:
  logger.error(f'{e}', exc_info=True)

df_train.head()



2024-07-26 21:53:44.561 | DEBUG    | __main__:<cell line: 3>:6 - DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'diagnosis', 'solutions'],
        num_rows: 476
    })
})
2024-07-26 21:53:44.568 | DEBUG    | __main__:<cell line: 3>:8 -       id                                               text  \
0    249  A cybersquatting domain save-russia[.]today is...   
1  14309  Like the Android Maikspy, it first sends a not...   
2  13996  While analyzing the technical details of this ...   
3  13600  (Note that Flash has been declared end-of-life...   
4  14364  Figure 21. Connection of Maikspy variants to 1...   
5  13565  The source code of this framework is shared ac...   
6   2400  The CVE-2022-22965 vulnerability allows an att...   
7  14126  At the same time, we were able to locate the M...   
8   1929  But this link is further fortified by the Andr...   
9   1921   A malicious RTF file that exploits the CVE-20...   

                        

,id,text,entities,relations,diagnosis,solutions
0,249,A cybersquatting domain save-russia[.]today is...,"[{'end_offset': 16, 'id': 44656, 'label': 'att...","[{'from_id': 44658, 'id': 9, 'to_id': 44659, '...",The diagnosis is a cyber attack that involves ...,1. Implementing DNS filtering to block access ...
1,14309,"Like the Android Maikspy, it first sends a not...","[{'end_offset': 17, 'id': 48530, 'label': 'SOF...","[{'from_id': 48531, 'id': 445, 'to_id': 48532,...",The diagnosis is that the entity identified as...,1. Implementing a robust anti-malware software...
2,13996,While analyzing the technical details of this ...,"[{'end_offset': 194, 'id': 48781, 'label': 'th...","[{'from_id': 48781, 'id': 461, 'to_id': 48782,...",Diagnosis: APT37/Reaper/Group 123 is responsib...,1. Implementing advanced threat detection tech...
3,13600,(Note that Flash has been declared end-of-life...,"[{'end_offset': 79, 'id': 51687, 'label': 'TIM...","[{'from_id': 51688, 'id': 1133, 'to_id': 51689...",The diagnosis is a malware infection. The enti...,1. Implementing a robust antivirus software th...
4,14364,Figure 21. Connection of Maikspy variants to 1...,"[{'end_offset': 191, 'id': 51779, 'label': 'UR...","[{'from_id': 51780, 'id': 1161, 'to_id': 44372...",The diagnosis is that Maikspy malware variants...,1. Implementing a robust firewall system that ...


# Data Cleaning and NLP

In [17]:
#DropNA and Duplicates
after_drop = df_train.dropna().drop_duplicates()
after_drop.shape
df_clean = after_drop.copy()
#assert before_drop == after_drop.shape

In [18]:
# Reduce to the observation (data), the diagnosis, and mitigations. Split out the entities array to
# mess around with graph based analysis later on.
pre_process = wandb.init(name="pre-processing cyber", project=project_name, job_type="pre-processing")
pre_data = wandb.Artifact(name="preprocessing_data", type="dataset")
pre_process.log_artifact(pre_data)



pre_process = wandb.Table(columns=["text", "diagnosis",  "solutions"])
pre_table = wandb.log({"table": df_train})


columns = ["text", "diagnosis", "solutions"]
df_train = df_train[columns]
wandb.log_artifact(pre_data, df_train)
wandb.save()



wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [19]:
# nlp cleaning
%pip install gensim nltk
import nltk
import gensim

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import random

def preprocess_text(text):
  # Stop words remove Tokenize the text
  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text.lower())
  filtered_sentence = [w for w in tokens if not w.lower() in stop_words and w.isalnum()]
  filterered_sentence=" ".join(filtered_sentence)

  return filtered_sentence




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# Tokenise and remove stopwords + Concatenate Scenario and Outcome
from tqdm import tqdm
import random
import sys
import time
from tqdm import tqdm_notebook
tqdm.pandas()
df_scenario_outcome = df_train.copy()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#logger.info(device)
try:
    # Apply preprocessing function with progress bar
    tqdm.pandas()
    df_scenario_outcome["text_pr"] = tqdm_notebook(df_scenario_outcome["text"].progress_apply(preprocess_text))
    df_scenario_outcome["diagnosis_pr"] = tqdm_notebook(df_scenario_outcome["diagnosis"].progress_apply(lambda x: preprocess_text(x) if x is not None else ''))
    df_scenario_outcome["solutions_pr "] = tqdm_notebook(df_scenario_outcome["solutions"].progress_apply(preprocess_text))
except Exception as e:
  #logger.error(e, exc_info=True)
  artifact = wandb.Artifact(name="pre_tokenisation", type="dataset")
  pre_data.add(df_scenario_outcome, "df_scenario_outcome")
  pre_data.log_artifact(artifact)
  wandb.run.save()

except Exception as e:
    print(f'Error: {e}', file=sys.stderr)
    sys.exit(1)





100%|██████████| 476/476 [00:00<00:00, 2740.53it/s]
<ipython-input-20-6f5c4067560b>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  df_scenario_outcome["text_pr"] = tqdm_notebook(df_scenario_outcome["text"].progress_apply(preprocess_text))


  0%|          | 0/476 [00:00<?, ?it/s]

100%|██████████| 476/476 [00:00<00:00, 2139.37it/s]
<ipython-input-20-6f5c4067560b>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  df_scenario_outcome["diagnosis_pr"] = tqdm_notebook(df_scenario_outcome["diagnosis"].progress_apply(lambda x: preprocess_text(x) if x is not None else ''))


  0%|          | 0/476 [00:00<?, ?it/s]

100%|██████████| 476/476 [00:00<00:00, 977.12it/s]
<ipython-input-20-6f5c4067560b>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  df_scenario_outcome["solutions_pr "] = tqdm_notebook(df_scenario_outcome["solutions"].progress_apply(preprocess_text))


  0%|          | 0/476 [00:00<?, ?it/s]

In [21]:
#Process Concat Field
df_scenario_outcome["scenario_outcome"] = df_scenario_outcome.progress_apply(
    lambda row: 'Scenario: ' + str(row["text"]) +'!!$$!!'+ 'Outcome: ' + str(row["diagnosis"]), axis=1)

100%|██████████| 476/476 [00:00<00:00, 81903.87it/s]


In [23]:
# Convert liss to strings
def list_to_string(text, index=1):
  text_pp = text.progress_apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
  return text_pp

In [24]:
df_str = df_scenario_outcome.copy()
for x in df_str:
  df_str.progress_apply(list_to_string, index=1)
  print(f"### {df_str[x]} processed###")
df_str.head()

100%|██████████| 476/476 [00:00<00:00, 385050.86it/s]

100%|██████████| 476/476 [00:00<00:00, 416977.59it/s]

100%|██████████| 476/476 [00:00<00:00, 504163.81it/s]

100%|██████████| 476/476 [00:00<00:00, 359987.14it/s]

100%|██████████| 476/476 [00:00<00:00, 376767.07it/s]

100%|██████████| 476/476 [00:00<00:00, 271964.13it/s]

100%|██████████| 7/7 [00:00<00:00, 183.40it/s]


### 0      A cybersquatting domain save-russia[.]today is...
1      Like the Android Maikspy, it first sends a not...
2      While analyzing the technical details of this ...
3      (Note that Flash has been declared end-of-life...
4      Figure 21. Connection of Maikspy variants to 1...
                             ...                        
471    Cyclops Blink, an advanced modular botnet that...
472     Sofacy Group has been associated with many at...
473    The plugin has been designed to drop multiple ...
474    We have uncovered a cyberespionage campaign be...
475    Based on the analysis of samples that were las...
Name: text, Length: 476, dtype: object processed###


100%|██████████| 476/476 [00:00<00:00, 399617.43it/s]

100%|██████████| 476/476 [00:00<00:00, 514029.02it/s]

100%|██████████| 476/476 [00:00<00:00, 344281.55it/s]

100%|██████████| 476/476 [00:00<00:00, 354931.33it/s]

100%|██████████| 476/476 [00:00<00:00, 274205.29it/s]

100%|██████████| 476/476 [00:00<00:00, 290229.50it/s]

100%|██████████| 7/7 [00:00<00:00, 173.12it/s]


### 0      The diagnosis is a cyber attack that involves ...
1      The diagnosis is that the entity identified as...
2      Diagnosis: APT37/Reaper/Group 123 is responsib...
3      The diagnosis is a malware infection. The enti...
4      The diagnosis is that Maikspy malware variants...
                             ...                        
471    Diagnosis: WatchGuard Firebox devices have bee...
472    The diagnosis is that Sofacy Group is a threat...
473    The infrastructure has been targeted by an att...
474    Diagnosis: Earth Baku, a known APT group with ...
475    The diagnosis is that the Maikspy variant is a...
Name: diagnosis, Length: 476, dtype: object processed###


100%|██████████| 476/476 [00:00<00:00, 539154.39it/s]

100%|██████████| 476/476 [00:00<00:00, 513764.46it/s]

100%|██████████| 476/476 [00:00<00:00, 551821.09it/s]

100%|██████████| 476/476 [00:00<00:00, 321237.12it/s]

100%|██████████| 476/476 [00:00<00:00, 408697.79it/s]

100%|██████████| 476/476 [00:00<00:00, 309725.21it/s]

100%|██████████| 7/7 [00:00<00:00, 185.52it/s]


...

100%|██████████| 476/476 [00:00<00:00, 463114.99it/s]

100%|██████████| 476/476 [00:00<00:00, 181978.74it/s]

100%|██████████| 476/476 [00:00<00:00, 448548.35it/s]

100%|██████████| 476/476 [00:00<00:00, 378553.03it/s]

100%|██████████| 476/476 [00:00<00:00, 233316.43it/s]

100%|██████████| 476/476 [00:00<00:00, 194646.46it/s]

100%|██████████| 7/7 [00:00<00:00, 152.48it/s]


### 0      [implementing, dns, filtering, block, access, ...
1      [implementing, robust, software, detect, remov...
2      [implementing, advanced, threat, detection, te...
3      [implementing, robust, antivirus, software, de...
4      [implementing, robust, firewall, system, detec...
                             ...                        
471    [update, watchguard, firebox, devices, latest,...
472    [implementing, comprehensive, threat, intellig...
473    [implementing, web, application, firewall, waf...
474    [implementing, network, segmentation, limit, l...
475    [educate, users, risks, downloading, apps, unt...
Name: solutions_pr , Length: 476, dtype: object processed###


100%|██████████| 476/476 [00:00<00:00, 278178.72it/s]

100%|██████████| 476/476 [00:00<00:00, 256354.48it/s]

100%|██████████| 476/476 [00:00<00:00, 405955.41it/s]

100%|██████████| 476/476 [00:00<00:00, 403983.95it/s]

100%|██████████| 476/476 [00:00<00:00, 332194.46it/s]

100%|██████████| 476/476 [00:00<00:00, 260570.18it/s]

100%|██████████| 7/7 [00:00<00:00, 158.21it/s]


### 0      Scenario: A cybersquatting domain save-russia[...
1      Scenario: Like the Android Maikspy, it first s...
2      Scenario: While analyzing the technical detail...
3      Scenario: (Note that Flash has been declared e...
4      Scenario: Figure 21. Connection of Maikspy var...
                             ...                        
471    Scenario: Cyclops Blink, an advanced modular b...
472    Scenario:  Sofacy Group has been associated wi...
473    Scenario: The plugin has been designed to drop...
474    Scenario: We have uncovered a cyberespionage c...
475    Scenario: Based on the analysis of samples tha...
Name: scenario_outcome, Length: 476, dtype: object processed###


,text,diagnosis,solutions,text_pr,diagnosis_pr,solutions_pr,scenario_outcome
0,A cybersquatting domain save-russia[.]today is...,The diagnosis is a cyber attack that involves ...,1. Implementing DNS filtering to block access ...,"[cybersquatting, domain, today, launching, dos...","[diagnosis, cyber, attack, involves, use, cybe...","[implementing, dns, filtering, block, access, ...",Scenario: A cybersquatting domain save-russia[...
1,"Like the Android Maikspy, it first sends a not...",The diagnosis is that the entity identified as...,1. Implementing a robust anti-malware software...,"[like, android, maikspy, first, sends, notific...","[diagnosis, entity, identified, software, infe...","[implementing, robust, software, detect, remov...","Scenario: Like the Android Maikspy, it first s..."
2,While analyzing the technical details of this ...,Diagnosis: APT37/Reaper/Group 123 is responsib...,1. Implementing advanced threat detection tech...,"[analyzing, technical, details, malware, inclu...","[diagnosis, 123, responsible, malware, attack,...","[implementing, advanced, threat, detection, te...",Scenario: While analyzing the technical detail...
3,(Note that Flash has been declared end-of-life...,The diagnosis is a malware infection. The enti...,1. Implementing a robust antivirus software th...,"[note, flash, declared, adobe, since, december...","[diagnosis, malware, infection, entity, affect...","[implementing, robust, antivirus, software, de...",Scenario: (Note that Flash has been declared e...
4,Figure 21. Connection of Maikspy variants to 1...,The diagnosis is that Maikspy malware variants...,1. Implementing a robust firewall system that ...,"[figure, connection, maikspy, variants, 107, 1...","[diagnosis, maikspy, malware, variants, commun...","[implementing, robust, firewall, system, detec...",Scenario: Figure 21. Connection of Maikspy var...


# Data as Artifact Artifact

In [25]:
# LOG TABLE
if wandb.run != None:
  wandb.finish()
  pass
save_table = wandb.init(job_type="dataset_raw", dir=f'{BASE_URL}{project_name}', name=f'raw-{model_name}')
raw_table = wandb.Table(dataframe=df_str, columns=columns)
save_table.log({"table": raw_table})
wandb.run.log({"table_key": raw_table})

# create a wandb Artifact for each meaningful step
cyber_data = wandb.Artifact("cyber-data-orion", type="dataset")

# [build up your predictions data as above]
cyber_data.add(raw_table, "cyber table with no additions")
save_table.log_artifact(cyber_data)



<Artifact cyber-data-orion>

In [161]:
'''import json
df_str.columns
df_str.isnull().drop(index=1, inplace=True)
df_str.dropna(inplace=True)
df_str.shape
df_str.columns

#export raw processed data CSV
file_name = f'/content/drive/MyDrive/models_datasets/datasets/{model_name}-csv-raw.csv'
df_str.to_csv(file_name)
print("CSV file created")
# JSONl
with open(file_name, 'w') as f:
  for entry in df_str:
    json.dump(entry, f)
    f.write('\n')

print('in JSON')



CSV file created
in JSON


In [134]:
### LOG TRAINING ARTEFACT ###
import random
import os

def init_run(project_name=project_name, job_type="training"):
  if wandb.run is None:
    new_run = f'run-{random.random():.2f}'
    print(new_run)
    wandb.init(project_name, new_run, job_type="training")
  else:
    wandb.run.finish()
    new_run = f'run-{random.random():.2f}'
    wandb.init(project_name, new_run, job_type="training")


def log_model_artifact(artifact_type, project=project_name, model_name=model_name, run=wandb.run):
  if wandb.run != None:
    wandb.run.finish()


  # Start a new W&B run

  if wandb.run is None:
    run = wandb.init(job_type="training", project=project)



  # Simulate logging model metrics
  run.log({"acc": random.random()}) #TODO ADD MORE METRICKS

  # Create a simulated model file
  with open(f"{model_name}.h5", "w") as f:
      f.write("Model: " + str(random.randomint()))

  # Log and link the model to the Model Registry
  run.link_model(path=f"({os.getenv('ROOT_DIR')}/{model_name}.h5", registered_model_name=f"{model_name}")

  # Finish the W&B run
  run.save()



# Row > Chat for OpenAI Format Function

In [27]:
'''Sentiment Analysis:
Added in some meta data to match the Scenario as outlined in the initial text column mapping it
to a scenario, and outcome. Then asking for the sentiment of the solutions'''
!pip install -qqq transformers torch accelerate torchvision
import wandb
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, pipeline

#uuid for files
import uuid
id = str(uuid.uuid4())[0:6]
dir = f"{BASE_URL}{project_name}/models/"
# Initialize W&B run for sentiment job
training_run = wandb.init(project=project_name, name=f"Sentiment_Analysis_{id}",
                 job_type="sentiment",dir=dir)


# Define model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL,)
#####

model = AutoModelForSequenceClassification.from_pretrained(MODEL, output_hidden_states=True)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
wandb.log({"object": classifier})
# Define paths and names
project = project_name
model_id = f"synOrion-cyber-{id}"
model_name = model_name
model_path = dir

# Save the model
check_point = classifier.save_pretrained(model_name)
wandb.save(check_point)

# Initialize a new W&B run to store the model


# Create a new artifact and add the model file to it
artifact = wandb.Artifact(name=f'{model_name}', type="model")
training_run.save()


object


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


True

In [ ]:
# Function to predict sentiment
import scipy


_logger.info(device)

def predict_sentiment(text=df_sentiment, model=model, classifier=classifier):
  model.to(device)
  if text is not None:
      labels = ["Negative", "Neutral", "Positive"]
      from scipy.special import softmax
      encoded_input = tokenizer(text, return_tensors='pt', truncation=True).to(device)
      # Run the model
      #with torch.no_grad(
      output = model(**encoded_input)
      # Extract the sentiment scores
      scores = output[0][0].detach().numpy()
      scores = softmax(scores)
      # Truncate the text to the maximum length the model can handle

      result = classifier(scores)
      ranking = np.argsort(result)
      ranking = ranking[::-1]
      for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")
      return sentiment, label

## Apply to copied DF
sentiment = df_sentiment['scenario_outcome'].progress_apply(predict_sentiment)
#score = df_sentiment["scenario_outcome"].progress_apply(predict_sentiment)
#df_sentiment['sentiment'] = sentiment[0]
#df_sentiment['score'] = sentiment[1]


In [174]:
df_chat = df_str.copy()
df_chat.drop(columns=["text", "diagnosis", "solutions"], inplace=True, index=1)
df_chat.head()



,text_pr,diagnosis_pr,solutions_pr,scenario_outcome
0,"[cybersquatting, domain, today, launching, dos...","[diagnosis, cyber, attack, involves, use, cybe...","[implementing, dns, filtering, block, access, ...",Scenario: A cybersquatting domain save-russia[...
2,"[analyzing, technical, details, malware, inclu...","[diagnosis, 123, responsible, malware, attack,...","[implementing, advanced, threat, detection, te...",Scenario: While analyzing the technical detail...
3,"[note, flash, declared, adobe, since, december...","[diagnosis, malware, infection, entity, affect...","[implementing, robust, antivirus, software, de...",Scenario: (Note that Flash has been declared e...
4,"[figure, connection, maikspy, variants, 107, 1...","[diagnosis, maikspy, malware, variants, commun...","[implementing, robust, firewall, system, detec...",Scenario: Figure 21. Connection of Maikspy var...
5,"[source, code, framework, shared, across, many...","[possible, diagnosis, data, leakage, explanati...","[implement, access, controls, restrict, access...",Scenario: The source code of this framework is...


In [167]:
for x in df_chat:
  df_chat.progress_apply(row_to_chat)
  print(f"### {df_chat[x]} processed###")
df_chat.head()

 14%|█▍        | 1/7 [00:00<00:00, 2090.88it/s]


KeyError: 'scenario_outcome'

In [ ]:
  #logger.error(e, exc_info=True)
  artifact = wandb.Artifact(name="chat structure", type="dataset")
  pre_data.add(df_chat, "df_scenario_outcome")
  pre_data.log_artifact(artifact)
  wandb.run.save()

# Final Processing Before Training
GPU requires the data & model to be on the GPU (or at least the same device if not GPU) REF: Mac torch.backends.mps.available() rather than cuda

In [135]:
### ENCODE CATEGORICAL VARIABLES
def preprocess_dataframe(dataframe):
    """
    Preprocess the DataFrame to ensure all data is numerical.
    Convert non-numeric columns to numerical data or drop them.
    """
    for x in dataframe.columns:
        if type(dataframe[x]) == list:
            dataframe[x] = dataframe[x].astype('category')
            dataframe[x] = dataframe[x].cat.codes
    # Convert categorical columns to numerical data
    df_encoded= pd.get_dummies(dataframe)
    logger.info("Converted categorical columns to numerical")

    # Ensure all data is float32
    df_encoded = df_encoded.astype(np.float32)
    logger.info("Converted all data to float32")

    return df_encoded


In [165]:
# Row to chat format function
def row_to_chat(row, index=1):
  return {
      "messages": [
          {
              "role": "system",
              "content": "You are a senior intelligence offficer. Assess the {scenario!!$$!!outcome} which determines the type of attack {solutions} provided. Then you will rate the solutions {0: low 1: medium 2: high in effectivenes}. Your answers will be in JSONl format."
          },
          {
              "role": "user",
              "content": row["scenario_outcome"]
          },
          {
              "role": "assistant",
              "content": f"""Scenario !!$$!! Context :: Sentiment:
                        {row["solution"]} !!$$!! {row["sentiment"]},"""
          }

          ]
  }

,text_pr,diagnosis_pr,solutions_pr,scenario_outcome
0,"[cybersquatting, domain, today, launching, dos...","[diagnosis, cyber, attack, involves, use, cybe...","[implementing, dns, filtering, block, access, ...",Scenario: A cybersquatting domain save-russia[...
1,"[like, android, maikspy, first, sends, notific...","[diagnosis, entity, identified, software, infe...","[implementing, robust, software, detect, remov...","Scenario: Like the Android Maikspy, it first s..."
2,"[analyzing, technical, details, malware, inclu...","[diagnosis, 123, responsible, malware, attack,...","[implementing, advanced, threat, detection, te...",Scenario: While analyzing the technical detail...
3,"[note, flash, declared, adobe, since, december...","[diagnosis, malware, infection, entity, affect...","[implementing, robust, antivirus, software, de...",Scenario: (Note that Flash has been declared e...
4,"[figure, connection, maikspy, variants, 107, 1...","[diagnosis, maikspy, malware, variants, commun...","[implementing, robust, firewall, system, detec...",Scenario: Figure 21. Connection of Maikspy var...


In [136]:
### ENCODE NOW
df_encoded = df_pp.copy()

df_encoded["enc_solutions"] = df_encoded["solutions_pr"].progress_apply(preprocess_text)
df_encoded.head()

KeyError: 'solutions_pr'

In [ ]:
"""
Utility function to convert a dataframe to a PyTorch tensor.
- More important with large datasets to be on the GPU
"""
# Set device to GPU if available, otherwise CPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
_logger.info(f"Data is on: {device}")


def to_numpy(dataframe):
    """
    Convert a DataFrame to a numpy array with float32 dtype.
    """
    try:
        df_numpy = dataframe.to_numpy(dtype=np.float32)
        _logger.info("Data is numpy array")
        _logger.info(f"Shape of numpy array: {df_numpy.shape}")
        return df_numpy
    except Exception as e:
        _logger.error(f'Error converting DataFrame to numpy array: {e}', exc_info=True)

def df_to_tensor(df_numpy, device=device):
    """
    Convert a numpy array to a PyTorch tensor and move it to the specified device.
    """
    try:
        df_tensors = torch.tensor(df_numpy, dtype=torch.float32)
        _logger.info("Data is PyTorch tensor with dtype torch.float32")
        df_tensors = df_tensors.to(device)
        _logger.info(f"Data is on {device}")
        return df_tensors
    except Exception as e:
        _logger.error(f'Error converting numpy array to PyTorch tensor: {e}', exc_info=True)


In [ ]:
# Convert DataFrame to numpy array
df_numpy = to_numpy(df_scenario_outcome)
logger.debug(df_numpy)

# Convert numpy array to PyTorch tensor
df_tensor = df_to_tensor(df_numpy)
logger.debug(df_tensor)

# Logging the shape of the tensor
logger.info(f"Shape of tensor: {df_tensor.shape}")

# Verify the type of df_tensor
logger.info(f"Type of df_tensor: {type(df_tensor)}")


In [ ]:
data_table = wandb.Table(dataframe=df_scenario_outcome, columns=["scenario_outcome", "solutions_pr"])

table_plot = run.plot_table(data_table=data_table,fields=["scenario_outcome","solutions"], vega_spec_name={project})
run.save()
plt.show(table_plot)
run.log({f"table_pot": f"{wandb.Graph(table_plot)}"})
wandb.save()

In [ ]:
# @title Model Registry { run: "auto" }
#huggingface_hub.login(token=HF_HUB, add_to_git_credential=True, write_permission=True)
# Start a new W&B run
run_name = f"{project}-save_model"

def check_run(run_name):
  if wandb.run is None:
    wandb.init(project=project, job_type="model", name=run_name)
  else:
    wandb.run.finish()
    wandb.init(project=project, job_type="model", name=run_name)








In [ ]:
def metrics_log(save_model: bool = False):
      wandb.run.name.log({"acc": acc})

    # Model File
    if os.path.exists(f"{BASE_URL}/{project}/models/") == False:
      os.mkdir(f"{BASE_URL}/{project}/models/")
      run.link_model(path=f"{BASE_URL}/{project}/models/{model_name}.h5", registered_model_name=model_name)
      run.save()
      with open(f"{BASE_URL}/{project}/models/{model_name}.h5", "w") as f:
        f.write("Model: " + str(random.random()))
      run.finish()

dedf64


In [ ]:
import matplotlib.pyplot as plt

df_sentiment = df_scenario_outcome.copy()
df_sentiment.drop(["text", "diagnosis", "solutions"], axis=1)
print(df_sentiment.shape)

#Make a data table from a dataframe
data_table = wandb.Table(dataframe=df_sentiment)
table_plot = wandb.plot_table(data_table=data_table,fields=["text","diagnosis","solutions"], vega_spec_name={project})
wandb.run.save()
plt.show(table_plot)
run.log({f"table_pot": f"{wandb.Graph(table_plot)}"})

In [ ]:
# prompt: Visualize in Weights and biases form everything until here

# Assuming 'run' is your active wandb run object

# Log summary metrics
wandb.run.summary["num_samples"] = len(df_scenario_outcome)
wandb.run.summary["num_unique_diagnoses"] = df_scenario_outcome["diagnosis"].nunique()
# Add other relevant summary stats

# Log the preprocessed dataframe as a table
preprocessed_table = wandb.Table(dataframe=df_pp,
                                 columns=["scenario_outcome", "solutions_pr"])
wandb.run.log({"preprocessed_data": preprocessed_table})

# Log the sentiment analysis results (assuming you have a df_sentiment with results)
sentiment_table = wandb.Table(dataframe=df_sentiment,
                               columns=["scenario_outcome", "solutions_pr", "sentiment"])
wandb.run.log({"sentiment_analysis_results": sentiment_table})

# Log the table plot you created earlier
wandb.run.log({"data_visualization": wandb.Image(table_plot)})

# Optionally log histograms of text lengths, word frequencies, etc.
# ...

# Finish the run
wandb.run.save()


In [ ]:
run.link_model(
    path=model_path,
    registered_model_name=f"{model_name}",
    name="4o-mini-cyber",
    aliases=["gpt4omini, finetune"],
)

### NER for enriching the data more ###

In [ ]:

artifact = wandb.Artifact(name=f'{model_name}', type="model")
artifact.add_file(local_path="./content/drive/MyDrive/models_datasets/models/", name=f'{local-path}-NER')
ner_model = 'dslim/bert-base-NER'
ner_tokenizer = AutoTokenizer.from_pretrained(ner_model)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model)

# Log the artifact to W&B
run.log_artifact(artifact)
huggingface_hub.save_pretrained_torch(model, model_name)
# Finish the W&B run
run.save()




In [ ]:
'''Sentiment Analysis:
Added in some meta data to match the Scenario as outlined in the initial text column mapping it
to a scenario, and outcome. Then asking for the sentiment of the solutions'''

import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoConfig

# Install necessary packages
!pip install -qqq transformers torch accelerate

# Initialize W&B run for sentiment job
training_run = wandb.init(project=project, name=f"Sentiment_Analysis_{id}",
                 job_type="sentiment",dir="/content/drive/MyDrive/models_datasets/models")


# Define model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
#####

model = AutoModelForSequenceClassification.from_pretrained(MODEL, output_hidden_states=True)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
wandb.log({"object": classifier})
# Define paths and names
project = project
model_id = "sentiment_model"
model_name = f"{project}-cyber{model_id}"
model_path = f"./content/drive/MyDrive/models_datasets/models/"

# Save the model
check_point = classifier.save_pretrained(model_name)
wandb.save(check_point)

# Initialize a new W&B run to store the model


# Create a new artifact and add the model file to it
artifact = wandb.Artifact(name=f'{model_name}', type="model")
training_run.save()


In [ ]:
import matplotlib.pyplot as plt

df_sentiment = df_scenario_outcome.copy()
df_sentiment.drop(["text", "diagnosis", "solutions"], axis=1)
print(df_sentiment.shape)

#Make a data table from a dataframe
data_table = wandb.Table(dataframe=df_sentiment)
table_plot = wandb.plot_table(data_table=data_table,fields=["text","diagnosis","solutions"], vega_spec_name={project})
wandb.run.save()
plt.show(table_plot)
run.log({f"table_pot": f"{wandb.Graph(table_plot)}"})

In [ ]:
### Text-Diagnosis Concatenation & HotEncoder Target
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
# FIX THIS
df_encoded = df_tokenized.copy()
df_encoded['text_diagnosis'] = df_text_diagnosis['text_processed'] + df_text_diagnosis['diagnosis_processed']

df_encoded.head()

# Simple Word to Vec Model

In [ ]:
# Train a Word2Vec model (example)
apply_word2Vec[column for column in columns]
model = Word2Vec(df_domains['text_processed'], min_count=1)

In [ ]:
# Apply the function to each column
df_w2v = df_domains.copy()
df_w2v['text_processed'] = df_domains['text'].apply(preprocess_text)
df_w2v['diagnosis_processed'] = df_domains['diagnosis'].apply(preprocess_text)
df_w2v['solutions_processed'] = df_domains['solutions'].apply(preprocess_text)

# OpenAI Embeddings with Gpt4o-Mini finely tuned and using the small OAI # Embeddings

In [ ]:
client = OpenAI()


In [ ]:

openai.api_key = userdata.get("OPENAI_API_KEY")
oai_model="text-embedding-3-small"

def get_openai_embedding(text,engine=oai_model):
    response = openai.Embedding.create(
      input=text,
      engine=engine  # Or another model you prefer
    )
    return response['text'][0]['embedding']



In [ ]:
embedded_data_oai.columns

In [ ]:
embedded_data_oai = df_encoded.copy()
embedded_data_oai['text_embedding'] = embedded_data_oai[''].progress_apply(get_openai_embedding)

In [ ]:
%pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns

# Start a new run for visualizations
viz_run = wandb.init(project="morpheus_cyber_gpt-4o-mini", job_type="visualization")

# --- Distribution of Sentiment ---
plt.figure(figsize=(8, 6))
sns.countplot(data=df_sentiment, x='sentiment')
plt.title('Distribution of Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
# Log the plot to W&B
viz_run.log({"sentiment_distribution": wandb.Image(plt)})
plt.show()

# --- Word Cloud of Text ---
from wordcloud import WordCloud
text_corpus = ' '.join(df_scenario_outcome['text_pr'].astype(str).tolist())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_corpus)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Text Data')
# Log the plot to W&B
viz_run.log({"text_wordcloud": wandb.Image(plt)})
plt.show()

# --- Word Cloud of Solutions ---
solutions_corpus = ' '.join(df_scenario_outcome['solutions_pr '].astype(str).tolist())
wordcloud_solutions = WordCloud(width=800, height=400, background_color='white').generate(solutions_corpus)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_solutions, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Solutions Data')
# Log the plot to W&B
viz_run.log({"solutions_wordcloud": wandb.Image(plt)})
plt.show()

# --- Length Distribution of Text ---
plt.figure(figsize=(8, 6))
sns.histplot(df_scenario_outcome['text_pr'].str.len(), bins=30)
plt.title('Distribution of Text Length')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
# Log the plot to W&B
viz_run.log({"text_length_distribution": wandb.Image(plt)})
plt.show()

# --- Correlation Heatmap (if applicable) ---
# If you have numerical features, you can create a correlation heatmap
# Example:
# numeric_features = df_scenario_outcome[['column1', 'column2']]  # Replace with actual numerical columns
# correlation_matrix = numeric_features.corr()
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
# plt.title('Correlation Heatmap')
# viz_run.log({"correlation_heatmap": wandb.Image(plt)})
# plt.show()

# --- Create an artifact and save the visualizations ---
artifact = wandb.Artifact(name="pre_finetuning_visualizations", type="visualizations")
# Add any files you want to include in the artifact (e.g., images, data files)
# artifact.add_file("path/to/your/file.png")

# Log the artifact to W&B
viz_run.log_artifact(artifact)

# Finish the visualization run
viz_run.finish()


# Preparing to Train
1. Isolate important columns.


In [ ]:
!pip install openai
import openai
openai.api_key = userdata.get("OPENAI_API_KEY")
from wandb.integration.openai.fine_tuning import WandbLogger
data=f"{BASE_URL}/{project}/{model_name}.jsonl"

from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("mydata.jsonl", "rb"),
  purpose="fine-tune"
)

# Finetuning logic
id = uuid.uuidv4()
if FINETUNE_JOB_ID == True:
  WandbLogger.sync(project=fine_tune_job_id=FINETUNE_JOB_ID)


WandbLogger.sync(entity="orion-agents-org")

# Fine Tuning Using Different Approaches
1. Open AI gpt-4o mini with small embeddings from OAI
2. Open AI gpt-4o mini with Word2Vec

1. Word2Vec model with Sentence Transformers

** After we have the models we will train them

In [100]:
### CHOOSE MODEL TO FINE TUNE
model = "gpt4o-mini"
###

prompt = f"{model}_prompt"

model_prompt = """
"messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions. 0 is bad, 1 is OK, 2, is GOOD"},
            {"role": "user", "content": {"scenario_outcome": "A ransomware attack encrypted critical files. Diagnosis: The attack vector was a phishing email. Solutions: 1. Isolate infected systems, 2. Pay the ransom, 3. Restore from backups."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Not advisable, 3. Good."}
"""



"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

messages: {
### {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."}
{}}
{}
### {"role": "user", "content": {"scenario_outcome": "A ransomware attack encrypted critical files. Diagnosis: The attack vector was a phishing email. Solutions: 1. Isolate infected systems, 2. Pay the ransom, 3. Restore from backups."}},
{}
###  {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Not advisable, 3. Good."}
{}}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("swaption2009/cyber-threat-intelligence-custom-data", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

NameError: name 'tokenizer' is not defined

In [ ]:
# prompt: train test split from sklearn 0.1 size , random_stat42

from sklearn.model_selection import train_test_split
train, test = train_test_split(df_train, test_size=0.1, random_state=42)


In [ ]:
### Need to concat the features.
#System Messages : 1 Assistant
```json
[
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "A ransomware attack encrypted critical files. Diagnosis: The attack vector was a phishing email. Solutions: 1. Isolate infected systems, 2. Pay the ransom, 3. Restore from backups."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Not advisable, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "Data breach exposing customer information. Diagnosis: Misconfigured cloud storage. Solutions: 1. Notify affected customers, 2. Implement stricter access controls, 3. Ignore the breach."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Bad."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "Unauthorized access to internal network. Diagnosis: Weak password policy. Solutions: 1. Change all passwords, 2. Implement MFA, 3. Monitor network traffic."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "DDoS attack causing service disruption. Diagnosis: Insufficient network defenses. Solutions: 1. Increase bandwidth, 2. Implement rate limiting, 3. Deploy DDoS protection service."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Partial, 2. Good, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "Malware infection on multiple devices. Diagnosis: Lack of antivirus software. Solutions: 1. Install antivirus software, 2. Perform a full system scan, 3. Disconnect infected devices from the network."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "Phishing attack leading to credential theft. Diagnosis: Lack of user training. Solutions: 1. Conduct phishing awareness training, 2. Change compromised credentials, 3. Implement email filtering."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "SQL injection attack compromising database. Diagnosis: Lack of input validation. Solutions: 1. Implement input validation, 2. Use parameterized queries, 3. Perform regular security audits."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "Unauthorized access to sensitive data. Diagnosis: Inadequate access controls. Solutions: 1. Restrict access to sensitive data, 2. Implement role-based access control, 3. Regularly review access logs."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "Insider threat leaking confidential information. Diagnosis: Lack of monitoring. Solutions: 1. Implement user activity monitoring, 2. Conduct background checks, 3. Establish a whistleblower policy."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Good."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "For each scenario, there is a scenario with a diagnosis and solutions. Decide if they are good solutions."},
            {"role": "user", "content": {"scenario_outcome": "Zero-day exploit used in an attack. Diagnosis: Outdated software. Solutions: 1. Apply patches promptly, 2. Use intrusion detection systems, 3. Maintain an incident response plan."}},
            {"role": "assistant", "content": "By my assessment, the solutions were: 1. Good, 2. Good, 3. Good."}
        ]
    }
]
```


In [ ]:
'''System Messages : Multiple Assistants
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris", "weight": 0}, {"role": "user", "content": "Can you be more sarcastic?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already.", "weight": 1}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "William Shakespeare", "weight": 0}, {"role": "user", "content": "Can you be more sarcastic?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?", "weight": 1}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "384,400 kilometers", "weight": 0}, {"role": "user", "content": "Can you be more sarcastic?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters.", "weight": 1}]}

In [ ]:
import json

#Define Template
system_messages = [{sys}]
prefix = {'messages':{"role:"system", "content": "Here are a variety of solutions to cyber problems. Analyze and give a binary 0 for no and 1 for yes.'}}}
postfix =

with open("./text_sql.json", "w") as f:
    json.dump(template, f)

In [ ]:
# DF to JSON Serialized
df_to_json = df_domain.to_json('./text_sql.json', orient='records')

In [ ]:
run = wandb.init(project="Cyber-Phi-Small-8k-instruct", job_type="dataset")
artifact = wandb.Artifact(name="df_to_json", type="dataset")
run.log_artifact(artifact)

run.finish()



---



## Model Training
1. Tokenize with TikToken
2. @ 4bit for improved speed traded off for lower precision calculation on weights.

In [ ]:
run2 = wandb.init(project="Cyber-Phi-Small-8k-instruct", job_type="train")
run3 = wandb.init(project="gpt4o-mini", job_type="train")

In [ ]:
# Load model directly from HuggingFace
%pip install -qq tiktoken einops
%pip install  -q torch==2.2.2+cu121 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu121 -U
_logger.info(device)
from unsloth import FastLanguageModel
import torch
import tiktoken
import einops
from transformers import AutoModelForCausalLM

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-small-8k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #token = os.getenv("WANDB_API_KEY"), # use one if using gated models like meta-llama/Llama-2-7b-hf
)

### Extract Entities for Graph
1. Make new dataFrame wth text an relations broken down, then labelled with Node or Relationship.


In [ ]:
import numpy as np
df_Graph = df_train.copy()
def graph_df(text):
  columns = text.unique()
  graph_df=pd.DataFrame(columns=columns)
  return graph_df
print(df_Graph.head())
graph_df = graph_df(df_Graph['entities'])
graph_df.shape
new_df = pd.DataFrame(columns=graph_df[0::])
new_df.head()
for k, v in enumerate(new_df.index):
  print(f'k is {k} and v is {v}')
  print(v)
  #new_df[f"{v}"] = df_Graph['entities'][k].split(',')
  #print(new_df[f"{v}]"])
  #print(graph_df.head())

#print(graph_df.describe)
#print(graph_df.head())
#print(np.array_split(values=graph_df,)

In [ ]:
def load_and_log():

    # 🚀 start a run, with a type to label it and a project it can call home
    with wandb.init(project="artifacts-data-models", job_type="load-data") as run:

        datasets = load()  # separate code for loading the datasets
        names = ["training", "validation", "test"]

        # 🏺 create our Artifact
        raw_data = wandb.Artifact(
            "cyber-phi", type="dataset",
            description="Cyber-Phi",
            metadata={"source": "torchvision.datasets.MNIST",
                      "sizes": [len(dataset) for dataset in datasets]})

        for name, data in zip(names, datasets):
            # 🐣 Store a new file in the artifact, and write something into its contents.
            with raw_data.new_file(name + ".pt", mode="wb") as file:
                x, y = data.tensors
                torch.save((x, y), file)

        # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)

load_and_log()



---
###APPENDIX A

### 🤗Fine-Tuning Techniques: 🤗

**PEFT** (Parameter-Efficient Fine-Tuning): Fine-tunes pre-trained models by adjusting only a small subset of parameters, reducing computational costs.

**LoRA** (Low-Rank Adaptation): Enhances transformer models by injecting and training low-rank matrices within each layer, minimizing the number of trainable parameters.

**QLoRA** (Quantized Low-Rank Adaptation): Combines low-rank adaptation with weight quantization to achieve efficient fine-tuning with reduced memory and computational requirements.

**Full Fine-Tuning:** Updates all parameters of the pre-trained model, offering high flexibility at the cost of increased computational resources.

**Distillation:** Trains a smaller model to mimic the behavior of a larger pre-trained model, optimizing efficiency while maintaining performance.